In [6]:
from pathlib import Path
import pandas as pd

# Base dir: la raíz del proyecto, subiendo 3 niveles desde el notebook
BASE_DIR = Path().resolve()
for _ in range(3):
    if (BASE_DIR / "data" / "clean" / "dataset_modelo.csv").exists():
        break
    BASE_DIR = BASE_DIR.parent

csv_path = BASE_DIR / "data" / "clean" / "dataset_modelo.csv"

try:
    df = pd.read_csv(csv_path, on_bad_lines='skip')
    print(f"✅ Dataset cargado correctamente con {len(df)} filas.")
except Exception as e:
    print(f"❌ Error al cargar el dataset: {e}")


✅ Dataset cargado correctamente con 8 filas.


In [7]:
with open(csv_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

print(f"👉 Total líneas (incluye cabecera): {len(lines)}")
print(f"✅ Líneas válidas: {len(df)}")
print(f"⚠️ Líneas con errores: {len(lines) - 1 - len(df)}")  # -1 por la cabecera


👉 Total líneas (incluye cabecera): 65
✅ Líneas válidas: 8
⚠️ Líneas con errores: 56


In [ ]:
# Leer líneas crudas
with open(csv_path, "r", encoding="utf-8") as f:
    lines = f.readlines()

# Extraer cabecera y contar columnas válidas
header = lines[0].strip().split(",")
expected_cols = len(header)

# Lista de filas válidas
rows_fixed = []

for i, line in enumerate(lines[1:], start=2):  # start=2 porque la línea 1 es cabecera
    fields = line.strip().split(",")

    if len(fields) > expected_cols:
        fields = fields[:expected_cols]  # Recorta columnas sobrantes
    elif len(fields) < expected_cols:
        continue  # ❌ Descarta si tiene menos columnas

    rows_fixed.append(fields)

# Crear DataFrame limpio
df_fixed = pd.DataFrame(rows_fixed, columns=header)

# Si hay columnas numéricas, intenta convertirlas
df_fixed = df_fixed.apply(pd.to_numeric, errors="ignore")
df_fixed["FECHA_EVENTO"] = pd.to_datetime(df_fixed["FECHA_EVENTO"], errors="coerce")

# Mostrar resumen
print(f"✅ Líneas reparadas: {len(df_fixed)} de {len(lines) - 1}")

# Mostrar las primeras filas
print(df_fixed.head())


✅ Líneas reparadas: 56 de 64
           NOMBRE_EVENTO FECHA_EVENTO COMUNIDAD  COSTE_UNITARIO  \
0  Sisterhood Premium 25   2021-05-14    GIRONA        4.561034   
1  Sisterhood Premium 30   2021-06-13    GIRONA        6.168077   
2  Sisterhood Premium 35   2021-07-18    GIRONA        3.878667   
3  Sisterhood Premium 40   2021-08-15    GIRONA        5.461053   
4  Sisterhood Premium 45   2021-09-05    GIRONA        6.202500   

   NUM_INSCRITAS  NUM_PAGOS  NUM_ASISTENCIAS  TOTAL_RECAUDADO  DIA_MES  \
0             29         25               25           295.25       14   
1             26         21               21           280.56       13   
2             30         27               27           286.74       18   
3             19         15               15           192.15       15   
4             28         21               21           207.27        5   

   SEMANA_DENTRO_DEL_MES  ...  COSTE_ESTIMADO BENEFICIO_ESTIMADO  \
0                      2  ...          132.27          

C:\Users\evaru\AppData\Local\Temp\ipykernel_8352\2621131270.py:26: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  df_fixed = df_fixed.apply(pd.to_numeric, errors="ignore")


In [18]:
from pathlib import Path

# Ruta relativa desde src/cleaning hasta la raíz del proyecto
salida = Path("../../data/clean/dataset_modelo.csv")
df_fixed.to_csv(salida, index=False)

In [24]:
from fuzzywuzzy import fuzz
import pandas as pd
from pathlib import Path

# === Rutas ===
eventos_path = Path("../../data/clean/dataset_modelo.csv")
instagram_path = Path("../../data/clean/instagram_media_text.csv")

# === Cargar datasets
df_eventos = pd.read_csv(eventos_path)
df_instagram = pd.read_csv(instagram_path)

# === Formateo
df_eventos["FECHA_EVENTO"] = pd.to_datetime(df_eventos["FECHA_EVENTO"], errors="coerce")
df_instagram["timestamp"] = pd.to_datetime(df_instagram["timestamp"], errors="coerce")
df_instagram["texto_completo"] = df_instagram["caption"].fillna('') + " " + df_instagram["ocr_text"].fillna('')
df_eventos["FECHA_EVENTO"] = df_eventos["FECHA_EVENTO"].dt.tz_localize(None)
df_instagram["timestamp"] = df_instagram["timestamp"].dt.tz_localize(None)

# === Filtro: solo eventos de pago
df_eventos_pago = df_eventos[df_eventos["TIPO_EVENTO"] == "pago"].copy()

# === Matching
shortcodes = []
fechas_post = []
scores = []

for _, evento in df_eventos_pago.iterrows():
    nombre_evento = evento["NOMBRE_EVENTO"].lower()
    fecha_evento = evento["FECHA_EVENTO"]

    mejor_score = 0
    mejor_shortcode = None
    mejor_fecha_post = None

    for _, post in df_instagram.iterrows():
        if post["timestamp"] > fecha_evento:
            continue  # Saltar posts posteriores al evento (recaps)

        texto = post["texto_completo"].lower()
        score = fuzz.partial_ratio(nombre_evento, texto)

        if score > mejor_score:
            mejor_score = score
            mejor_shortcode = post["shortcode"]
            mejor_fecha_post = post["timestamp"]

    shortcodes.append(mejor_shortcode)
    fechas_post.append(mejor_fecha_post)
    scores.append(mejor_score)

# === Añadir al dataframe
df_eventos_pago["INSTAGRAM_SHORTCODE"] = shortcodes
df_eventos_pago["FECHA_POST"] = fechas_post
df_eventos_pago["DIF_DIAS_PUBLICACION"] = (df_eventos_pago["FECHA_EVENTO"] - df_eventos_pago["FECHA_POST"]).dt.days
df_eventos_pago["MATCH_SCORE"] = scores

# === Mostrar preview
print(df_eventos_pago[["NOMBRE_EVENTO", "FECHA_EVENTO", "INSTAGRAM_SHORTCODE", "FECHA_POST", "DIF_DIAS_PUBLICACION", "MATCH_SCORE"]])


                       NOMBRE_EVENTO FECHA_EVENTO  INSTAGRAM_SHORTCODE  \
0    Afterwork Run By Dash And Stars   2025-04-17                  NaN   
1                 Dash And Stars Run   2025-02-09                  NaN   
73             New Year Runbreakfast   2025-01-12                  NaN   
74              Planazo Pre Vacances   2024-07-21                  NaN   
75          Pre Xmas Runbrunch Elche   2024-12-22                  NaN   
79  Rundeo By Primetime I Sisterhood   2025-05-09                  NaN   
80                          Runlearn   2025-04-06                  NaN   
81                          Runtrust   2025-03-09                  NaN   
82            Sisterhood Codepilates   2024-10-06                  NaN   
85             Sisterhood Xmas Event   2024-12-15                  NaN   
92                        Sunday Run   2025-03-09                  NaN   
94   Sunkissed Run By Dash And Stars   2025-06-08                  NaN   

            FECHA_POST  DIF_DIAS_PUBL

In [25]:
from fuzzywuzzy import fuzz
import pandas as pd
from pathlib import Path

# === Rutas
eventos_path = Path("../../data/clean/dataset_modelo.csv")
instagram_path = Path("../../data/clean/instagram_media_text.csv")

# === Cargar datasets
df_eventos = pd.read_csv(eventos_path)
df_instagram = pd.read_csv(instagram_path)

# === Formatear fechas y textos
df_eventos["FECHA_EVENTO"] = pd.to_datetime(df_eventos["FECHA_EVENTO"], errors="coerce").dt.tz_localize(None)
df_instagram["timestamp"] = pd.to_datetime(df_instagram["timestamp"], errors="coerce").dt.tz_localize(None)
df_instagram["texto_completo"] = df_instagram["caption"].fillna('') + " " + df_instagram["ocr_text"].fillna('')

# === Filtrar eventos de pago
df_eventos_pago = df_eventos[df_eventos["TIPO_EVENTO"] == "pago"].copy()

# === Matching
resultados = []

for _, evento in df_eventos_pago.iterrows():
    nombre_evento = evento["NOMBRE_EVENTO"].lower()
    fecha_evento = evento["FECHA_EVENTO"]

    # Filtrar posts anteriores o el mismo día
    df_posts_validos = df_instagram[df_instagram["timestamp"] <= fecha_evento].copy()
    df_posts_validos["score"] = df_posts_validos["texto_completo"].str.lower().apply(
        lambda texto: fuzz.partial_ratio(nombre_evento, texto)
    )

    df_posts_validos = df_posts_validos[df_posts_validos["score"] >= 60]

    if not df_posts_validos.empty:
        mejor_post = df_posts_validos.sort_values("timestamp", ascending=False).iloc[0]
        resultados.append({
            "NOMBRE_EVENTO": evento["NOMBRE_EVENTO"],
            "FECHA_EVENTO": fecha_evento,
            "INSTAGRAM_SHORTCODE": mejor_post["shortcode"],
            "FECHA_POST": mejor_post["timestamp"],
            "MATCH_SCORE": mejor_post["score"],
            "DIF_DIAS_PUBLICACION": (fecha_evento - mejor_post["timestamp"]).days
        })
    else:
        resultados.append({
            "NOMBRE_EVENTO": evento["NOMBRE_EVENTO"],
            "FECHA_EVENTO": fecha_evento,
            "INSTAGRAM_SHORTCODE": None,
            "FECHA_POST": None,
            "MATCH_SCORE": 0,
            "DIF_DIAS_PUBLICACION": None
        })

# === Crear DataFrame resultado
df_resultado = pd.DataFrame(resultados)

# === Vista previa
print(df_resultado.head(15))


                       NOMBRE_EVENTO FECHA_EVENTO  INSTAGRAM_SHORTCODE  \
0    Afterwork Run By Dash And Stars   2025-04-17                  NaN   
1                 Dash And Stars Run   2025-02-09                  NaN   
2              New Year Runbreakfast   2025-01-12                  NaN   
3               Planazo Pre Vacances   2024-07-21                  NaN   
4           Pre Xmas Runbrunch Elche   2024-12-22                  NaN   
5   Rundeo By Primetime I Sisterhood   2025-05-09                  NaN   
6                           Runlearn   2025-04-06                  NaN   
7                           Runtrust   2025-03-09                  NaN   
8             Sisterhood Codepilates   2024-10-06                  NaN   
9              Sisterhood Xmas Event   2024-12-15                  NaN   
10                        Sunday Run   2025-03-09                  NaN   
11   Sunkissed Run By Dash And Stars   2025-06-08                  NaN   

            FECHA_POST  MATCH_SCORE  

In [30]:
# Detectar base del proyecto de forma robusta
try:
    BASE_DIR = Path(__file__).resolve().parents[2]
except NameError:
    BASE_DIR = Path(os.getcwd())  # Para entornos como Streamlit o notebooks

# Ruta al CSV
csv_path = BASE_DIR / "data/clean/dataset_modelo.csv"

# Cargar dataset
df = pd.read_csv(csv_path)

# Eliminar columnas relacionadas con Instagram si existen
columnas_instagram = [
    'FECHA_POSTEO_IG',
    'SHORTCODE_IG',
    'URL_IMAGEN_IG',
    'FECHA_DET_PREDICHA'
]
df.drop(columns=columnas_instagram, errors='ignore', inplace=True)

# Guardar dataset limpio
df.to_csv(csv_path, index=False)
print("✅ Dataset limpio guardado sin columnas de Instagram.")


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\evaru\\OneDrive\\Escritorio\\sismiaio\\src\\cleaning\\data\\clean\\dataset_modelo.csv'